# Machine Learning 3rd Assignment Report

## Part II

### What
We are building a neural network and neural network interface to classify images of flowers. For that reason we are creating single and multi layer networks.

### How
Let's talk about what steps I followed for this problem. 
#### Structure


Firstly I had to create project structure. I created a python package for neural network interface and one for data. The data package was ready anyway so I just added methods to load the files. For neural network I wanted to create a dynamic network so I implemented a json loader method. From json template one can create different kinds of network similar to the ones in caffe.
#### Preparing data
Data is vectorized greyscale images of certain flower types. That handled most of the job. I took image vectors then squashed their intensity values between 0 and 1. Then I hot encoded their labels so I can classify them.
I did same thing for validation data so that I can validate as the model trained.
#### Regularization
I have tried few regularization methods like __l1__, __l2__ and __dropout__ regularization. None of them worked out well. Perhaps I failed to implement them or maybe model wasn't suitable for this kind of task. Anyway, I only left dropout since I implemented others pretty early on in a desperate attempt to improve the validation scores. When I saw that they actually worsened the situtation(underflows) I removed them and never looked back. At this point I believe it is just waste of time to reimplement them since they do nothing positive on the model. <br>
Let's talk about dropout specifically. It is the only regularization required a seperate method, meaning I could turn it on and off at will without much work. I added dropout method to the layers and it is called during forward pass after activation functions called. Neural network has a parameter called dropout_probabilty which used during dropout sequence. I used 0.5 mostly and never really tried anything else. During training with dropout, obviously, training scores drops significantly since half of the neurons not working during prediction while validation accuracy does not change a lot. 
#### Losses and Activations
I put every loss and activation function on seperate file called 'math_'. 
List of currently available activation functions:
 - __Sigmoid__: You can't go wrong with sigmoid. I polished it a lot so it is best implemented one out of the rest.
 - __Tanh__: Gives similar result to sigmoid. 
 - __Softmax__: Only used on output layer. Works pretty fine.
 - __ReLu__: Problem child. Because of dying relu problem you will quickly get all 0 outputs.
 - __Leaky ReLu__: In an attempt to solve the dying relu problem I added this. Never got it to work because of the overflows. Can't really care at this point.
 
I also have method for signum but I did not test it so you can't use it. <br>
List of currently available loss functions:

 - __Cross entropy__: First added loss function. Works just fine.
 - __Sum of squares__: Works ok, but did not tested it extensively.
 - __Mean squared error__: Implementation is derived from sum of squares and derivative calculated by me. So possibly does not work correctly.
 - __R Squared Error__: I added this last. While I was working with scikit learn I saw this and tried to implement it myself. Currently gives only negative output somehow so I suggest not using it.
 
Moral of the story, stick to the cross entropy.

### Analyses

#### Analysis of activation functions

#### Analysis of Loss functions

#### Analysis of dropout regularization

#### Analysis of training/validation scores

#### Analysis of momentum

#### <center>Analysis of activation functions</center>

|Parameter|Value|
|-|-|
|Hidden layer count|1|
|Hidden layer node count|40|
|Output activation|Softmax|
|Loss|Cross entropy|
|Epochs|300|
|Batch size|5|
|Learning rate bound|0.02 - 0.0005|
|Decay rate|0.0005|
|Dropout chance|0.0|
|Momentum parameter|1.0|


 - __Sigmoid:__
    > ![sigmoid](plots/layers/1_hidden_40_node_no_dropout.png)
    
 - __Tanh:__
    > ![sigmoid](plots/layers/1_hidden_40_node_no_dropout.png)

#### <center>Analysis of layer count</center>

|Parameter|Value|
|-|-|
|Hidden activation|Sigmoid|
|Output activation|Softmax|
|Loss|Cross entropy|
|Epochs|300|
|Batch size|5|
|Learning rate bound|0.02 - 0.0005|
|Decay rate|0.0005|
|Dropout chance|0.0|
|Momentum parameter|1.0|

- __Single layer:__
> ![single_layer](plots/layers/single_layer_no_dropout.png)

- __One hidden layer with 40 nodes:__
> ![10node1hidden](plots/layers/1_hidden_40_node_no_dropout.png)

- __One hidden layer with 100 nodes:__
> ![100node1hidden](plots/layers/1_hidden_100_node_no_dropout.png)

- __Two hidden layers with 40 and 40 nodes each:__
> ![40_40node2hidden](plots/layers/2_hidden_40_40_node_no_dropout.png)
 
> The graphs above selected because we have:
- Single layer sample
- One sample with single hidden layer
- One sample with single hidden layer but with increased node count
- One sample with two hidden layers with node counts same as the single hidden layer one <br>

> These tells us what we need to see:
- __Looking at loss__ we see that results are improving as we increse node counts but not the layer count. Single layer was enough to solve our problem but we may need to increase the number of nodes.
- __Looking at training accuracy__ we see again that we only need single layer but with larger number of nodes. Even then, results are close for all tests since overall accuracy is low.
- __Looking at validation accuracy__ we now see the best solution. While higher node counts leads to incresed training accuracy the gap between it and validation accuracy increases. As the layer count and node count increses training accuracy starting to overshoot the validation accuracy. That shows as our model is starting to overfit at that point.

From these results we can conclude that best possible model comes from _lower hidden layer count_ with _average node count._ Single layer simply wasn't enough for our data. 
  

#### <center>Analysis of activation functions</center>